In [141]:
#Integrantes:
#-Mendez Hilario Ricardo Camilo 20191412
#-Saavedra Quispe Daniel Salvador 20200336
#-Vargas Cespedes Jose Enmanuel 20191317

In [142]:
# Solicitamos las entradas necesarias para que el programa brinde la informacion requerida
# Ejemplo de productos que se puede solicitar con el programa: arroz, azúcar, aceite, leche, etc.
print("INGRESE EL NOMBRE DEL PRODUCTO DE LA SIGUIENTE FORMA PRODUCTO+MARCA+PREFERENCIA")
producto=input("ingrese el producto que desee buscar: ") #solicitamos el nombre del producto
marca=input("ingrese la marca: ") #especificamos la marca del producto
nombre_comun=input("ingrese el nombre en comun del producto: ") #especificamos el nombre del producto
palabra_clave=input("ingrese una palabra clave por ejemplo: si busca lecha sin lactosa = la palbra clave sera lactosa: ") #especificamos si el producto tiene algun tipo de variación
destinatario=input("ingrese el correo de destino: ") # se solicita el correo de la persona a la cual le llegara los resultados
filtros=[marca,nombre_comun,palabra_clave] #establecemos una lista que servira de filtro para que en la tabla final, el programa brinde correctamente la información. 


INGRESE EL NOMBRE DEL PRODUCTO DE LA SIGUIENTE FORMA PRODUCTO+MARCA+PREFERENCIA
ingrese el producto que desee buscar: pan molde 
ingrese la marca: bimbo
ingrese el nombre en comun del producto: pan
ingrese una palabra clave por ejemplo: si busca lecha sin lactosa = la palbra clave sera lactosa: pan
ingrese el correo de destino: anavargas@lamolina.edu.pe


In [143]:
destinatario

'anavargas@lamolina.edu.pe'

In [144]:
urls=["https://www.metro.pe/","https://www.plazavea.com.pe/","https://www.tottus.com.pe/"]   # url de la pagina principal de los supermercados METRO,  PLAZA VEA , TOTTUS

In [145]:
super_mercados=["metro","plazavea","tottus"] #Lista que contiene los nombres de los supermercados

for i in super_mercados: #Generamos este lazo que permitira formar la url correcta de la variedad de opciones que hay para el producto solicitado por cada pagina de supermercado
  if i=="metro":
    urlb_metro=urls[0]+"busca/?ft="+producto.replace(" ", "%20") #Se descubrieron patrones en los urls de las paginas, ejem (metro): comienza con el url de la pagina principal seguido de un "busca/?ft=" mas el nombre del producto
  if i=="plazavea":
     urlb_plazavea=urls[1]+"search/?_query="+producto.replace(" ", "%20")
  if i=="tottus":
    urlb_tottus=urls[2]+"buscar?q="+producto.replace(" ", "%20")


#**CODIGO METRO**

In [ ]:
!apt-get update #actualizamos todos los paquetes qu ya se encuentran instalados
!apt install chromium-chromedriver #instalamos el driver de chrome, este driver es necesario para que selenium pueda funcionar 
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium #instalamos selenium

In [147]:
from selenium import webdriver #cargamos la funcion que nos permite hacer uso del driver de selenium

options = webdriver.ChromeOptions() #establecemos algunas opciones del driver
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=options) #Instanciamos el driver de selenium que va a controlar el navegador
driver.implicitly_wait(30) # indicamos el tiempo (en milisegundos) para que el controlador arroje los resultados luego de hacer la busqueda del elemento.

In [148]:
#PROBANDO PARA METRO
driver.get(urlb_metro) #sirve para ir a la pagina que queremos scrapear

In [149]:
urlb_metro

'https://www.metro.pe/busca/?ft=pan%20molde%20'

In [150]:
from selenium.webdriver.common.by import By #nos sirve para establecer un Conjunto de estrategias de localización compatibles.

In [151]:
driver.execute_script("window.scrollTo(0,document.body.scrollHeight)") # permite desplazarse hasta el final de la página (pagina con carga dinámica)

In [152]:
import requests # importamos http
from bs4 import BeautifulSoup #Para parsear el arbol HTML

In [153]:
soup_metro=BeautifulSoup(driver.page_source,"html.parser") #Nos arroja un arbol HTML más legible.

In [154]:
name_metro = soup_metro.find("div", class_="product-shelf").find_all('a', class_ ='product-item__name') #Hallamos todos los hijos "a" del tag "div" con class="product-shelf", en estos tags "a" se encuentran los nombres de todos los productos de la pagina que coninciden con el producto solicitado
precios_metro=soup_metro.find("div", class_="product-shelf").find_all('span', class_ ='product-prices__value product-prices__value--best-price') # lo mismo que en la variable "name_meto" pero esta vez con los precios
nombres_productos_metro=[] #lista vacia que contendra todos los nombres hallados
precios_producto_metro=[] #lista vacia que contendra todos los prcios hallados

In [155]:
len(name_metro)

30

In [156]:
import re #libreria de expresiones regulares

In [157]:
import pandas as pd 
for i in range(len(name_metro)): #lazo o bucle que agrega cada nombre y precio encontrado a las listas vacias
  nombres_productos_metro.append(name_metro[i].text) #agregamos solo el texto a la lista vacia nombre_productos_metro (en este caso solo los nombres)
  precios_producto_metro.append(precios_metro[i].text)

precios_metro_formato= [float(re.findall(r'-?\d+\.?\d*', i)[0]) for i in precios_producto_metro] # arreglamos el formato del precio para que nos arroje solo el valor numerico
precios_metro_formato
mercado=["metro"]*len(precios_metro_formato) #nueva columna que indicara que los productos de esta tabla pertenecen al supermercado metro

In [158]:
df_metro=pd.DataFrame({"Nombre_producto":nombres_productos_metro,"Precios_en_soles":precios_metro_formato, "Supermercado":mercado }) #creamos un dataframe que contendra todas las listas desarrolladas para el supermecado metro

In [159]:
df_metro

,Nombre_producto,Precios_en_soles,Supermercado
0,Pan de Molde Blanco Bimbo 480g,11.20,metro
1,Pan de Molde Integral Bimbo 480g,11.20,metro
2,Pan de Molde Unión Multisemillas con Avena 540g,7.80,metro
3,Pan de Molde Integral Bimbo Bolsa 750 g,14.80,metro
4,Pan de Molde Blanco Bimbo Bolsa 750 g,14.80,metro
5,Pan de Molde Multigranos Misturapan Bolsa 390 g,11.59,metro
6,Pan de Molde de Yema Misturapan Bolsa 390 g,11.59,metro
7,Pan de Molde Campesino Misturapan Bolsa 390 g,11.59,metro
8,Pan de Molde Blanco Familiar PYC Bolsa 500 g,9.70,metro
9,Pan de Molde Unión Multisemilla & Chips Miel b...,8.63,metro


#**PLAZA VEA**

In [160]:
driver.get(urlb_plazavea) #sirve para ir a la pagina que queremos scrapear

In [161]:
urlb_plazavea

'https://www.plazavea.com.pe/search/?_query=pan%20molde%20'

In [162]:
from selenium.webdriver.common.by import By #nos sirve para establecer un Conjunto de estrategias de localización compatibles.

In [163]:
gs = driver.find_elements(By.CLASS_NAME, "Showcase__name") #almacena los elementos que contengan class = "Showcase__name"
nombre_plasavea=[] #lista vacia que contendra los nombres de los productos brindados por plazavea
for elemento in gs:
  nombre_plasavea.append(elemento.text) #Agregamos el texto de cada elemento hallado y lo almacenamos en la lista vacia

nombre_plasavea
len(nombre_plasavea)

40

In [164]:
import re

In [165]:
gss=driver.find_elements(By.XPATH,"//div[@class='Showcase__salePrice']") #A traves de XPATH buscamos todos los tags "div" cuyo class sea igual a "Showcase_salePrice"
precios_plazavea=[] #lista vacia que contendra los precios de los productos
for elemento in gss:
  sentence = elemento.text #extraemos el texto por cada elemento
  s=re.findall(r'-?\d+\.?\d*', sentence) # extraemos solo el valor numerico
  s=float(s[0]) #transformamos a tipo flotante o decimal
  precios_plazavea.append(s) #lo agregamos a la lista vacia precios
len(precios_plazavea)

40

In [ ]:
precios_plazavea

In [167]:
df_plazavea=pd.DataFrame({"Nombre_producto":nombre_plasavea,  "Precios_en_soles":precios_plazavea, "Supermercado":["plazavea"]*len(nombre_plasavea)}) #creamos un dataframe que contendra todas las listas desarrolladas para el supermecado plazavea


In [168]:
df_plazavea

,Nombre_producto,Precios_en_soles,Supermercado
0,Pan de Molde Blanco BIMBO Extra Grande Bolsa 7...,14.50,plazavea
1,Pan de Molde BIMBO Integral Bolsa 750g,14.70,plazavea
2,Pan de Molde Integral BIMBO Vital Multicereal ...,14.80,plazavea
3,Pan de Molde Blanco BIMBO Artesano Bolsa 530g,12.40,plazavea
4,Pan de Molde Blanco BIMBO sin Bordes Bolsa 650...,15.30,plazavea
5,Pan de Molde Blanco PYC Familiar Bolsa 500g,9.60,plazavea
6,Pan de Molde Blanco DON MAMINO sin Corteza Bol...,13.60,plazavea
7,Pan de Molde Integral BELL'S Grande Bolsa 800g,11.89,plazavea
8,Pan de Molde UNIÓN Granos Integrales Bolsa 540...,10.30,plazavea
9,Pan de Molde Blanco BIMBO Vital de Semillas Bo...,14.80,plazavea


#**TOTTUS**

In [169]:
#CODIGO PARA TOTTUS
urlb_tottus

'https://www.tottus.com.pe/buscar?q=pan%20molde%20'

In [170]:
driver.get(urlb_tottus) #sirve para ir a la pagina que queremos scrapear

In [171]:
from selenium.webdriver.common.by import By

In [172]:
gs_tottus = driver.find_elements(By.XPATH,"//h2[@class='jsx-369422875 name title mini']")  #A traves de XPATH buscamos todos los tags "h2" cuyo class sea igual a "jsx-369422875 name title mini"
nombre_tottus=[] #lista vacia que contendra los nombres de los productos brindados por tottus
for elemento in gs_tottus:
  nombre_tottus.append(elemento.text) #Agregamos el texto de cada elemento hallado y lo almacenamos en la lista vacia

nombre_tottus

['PAN MOLDE BLANCO TOTTUS 600 GR',
 'PAN MOLDE INTEGRAL TOTTUS 600 GR',
 'PAN MOLDE BLANCO ARTESANO BIMBO 530 GR',
 'PAN MOLDE MARMOL SIN CORTE DON MAMINO 560 GR',
 'PAN MOLDE BLANCO SIN CORTEZA DON MAMINO 560 GR',
 'PAN DE MOLDE BLANCO SIN CORTEZA 540 GR',
 'PAN MOLDE CON SALVADO DE TRIGO SIN CORTEZA TOTTUS',
 'PAN DE MOLDE BLANCO DON MAMINO 560 GR',
 'PAN DE MOLDE INTEGRAL TOTTUS 500 GR',
 'PAN DE MOLDE BLANCO TOTTUS 500 GR',
 'PAN DE MOLDE BLANCO PYC 500 GR',
 'PAN DE MOLDE INTEGRAL PYC 500 GR',
 'PAN BIMBO MOLDE REBANADO FUENTE DE CALCIO 480 GR',
 'PAN FIBRA UNIÓN 540 GR',
 'PAN CON GRANOS INTEGRALES UNIÓN540 GR',
 'PAN BLANCO BIMBO 750 GR',
 'PAN DOBLE FIBRA BIMBO 420 GR',
 'PAN INTEGRAL SIN CORTEZA BIMBO 650 GR',
 'PAN BIMBO BLANCO CON SEMILLAS 600 GR',
 'PAN INTEGRAL MULTICEREAL BIMBO 600 GR',
 'PAN HAMBURGUESA TOTTUS 8 UNIDADES',
 'PAN CIABATTA PRECOCIDO DON MAMINO 250 GR',
 'PAN INTEGRAL EXTRA GRANDE BIMBO 750 GR',
 'PAN MULTICEREAL VITAL BIMBO 360 GR',
 'PAN ARTESANO INTEGRAL

In [173]:
len(nombre_tottus)

46

In [ ]:
gs_tottus_precios = driver.find_elements(By.XPATH,"//span[@class='list price medium cmrPrice ']") #A traves de XPATH buscamos todos los tags "span" cuyo class sea igual a "list price medium cmrPrice"
precios_tottus=[] #lista vacia que contendra los precios de los productos
for elemento in gs_tottus_precios:
  sentence = elemento.text #extraemos el texto por cada elemento
  s=re.findall(r'-?\d+\.?\d*', sentence) # extraemos solo el valor numerico
  s=float(s[0]) #transformamos a tipo flotante o decimal
  precios_tottus.append(s) #lo agregamos a la lista vacia precios
precios_tottus

In [175]:
len(precios_tottus)

46

In [176]:
df_tottus=pd.DataFrame({"Nombre_producto":nombre_tottus,"Precios_en_soles":precios_tottus, "Supermercado":["tottus"]*len(precios_tottus) })#creamos un dataframe que contendra todas las listas desarrolladas para el supermecado tottus

In [177]:
df_tottus

,Nombre_producto,Precios_en_soles,Supermercado
0,PAN MOLDE BLANCO TOTTUS 600 GR,8.9,tottus
1,PAN MOLDE INTEGRAL TOTTUS 600 GR,8.9,tottus
2,PAN MOLDE BLANCO ARTESANO BIMBO 530 GR,12.0,tottus
3,PAN MOLDE MARMOL SIN CORTE DON MAMINO 560 GR,12.2,tottus
4,PAN MOLDE BLANCO SIN CORTEZA DON MAMINO 560 GR,13.5,tottus
5,PAN DE MOLDE BLANCO SIN CORTEZA 540 GR,11.7,tottus
6,PAN MOLDE CON SALVADO DE TRIGO SIN CORTEZA TOTTUS,11.7,tottus
7,PAN DE MOLDE BLANCO DON MAMINO 560 GR,12.2,tottus
8,PAN DE MOLDE INTEGRAL TOTTUS 500 GR,8.9,tottus
9,PAN DE MOLDE BLANCO TOTTUS 500 GR,8.9,tottus


## Juntando salidas (METRO, PLAZAVEA, TOTTUS)

In [178]:
# importamos las librerias necesarias para manipular y crear los dataframes
import numpy as np
import pandas as pd

In [179]:
df_total=pd.concat([df_tottus, df_plazavea, df_metro]) # juntamos los dataframes de cada supermercado en un solo dataframe. 

In [180]:
len(df_tottus) #mostramos por cuantas filas esta compuesta la tabla de tottus

46

In [181]:
len(df_plazavea) #mostramos por cuantas filas esta compuesta la tabla de plazavea 

40

In [182]:
len(df_metro) #mostramos por cuantas filas esta compuesta la tabla de metro

30

In [183]:
df_total=df_total.reset_index() #reseteamos el index de nuestra tabla total

In [184]:
len(df_total)

116

In [185]:
df_total=df_total.drop(columns=['index']) #eliminamos la columna "index" 

In [186]:
df_total

,Nombre_producto,Precios_en_soles,Supermercado
0,PAN MOLDE BLANCO TOTTUS 600 GR,8.9,tottus
1,PAN MOLDE INTEGRAL TOTTUS 600 GR,8.9,tottus
2,PAN MOLDE BLANCO ARTESANO BIMBO 530 GR,12.0,tottus
3,PAN MOLDE MARMOL SIN CORTE DON MAMINO 560 GR,12.2,tottus
4,PAN MOLDE BLANCO SIN CORTEZA DON MAMINO 560 GR,13.5,tottus
...,...,...,...
111,Pan de Molde Doble Fibra Bimbo Bolsa 420 g,11.2,metro
112,Pan de Molde de Mix de Cereales Mestemacher Bo...,14.9,metro
113,Pan de Molde con Semillas de Girasol Mestemach...,14.9,metro
114,Pan de Molde Integral con Frutas y Cereales Bi...,10.9,metro


In [187]:
df_total["Nombre_producto"][0]

'PAN MOLDE BLANCO TOTTUS 600 GR'

In [188]:
import pandas as pd 

In [189]:
filtros

['bimbo', 'pan', 'pan']

In [190]:
df_total

,Nombre_producto,Precios_en_soles,Supermercado
0,PAN MOLDE BLANCO TOTTUS 600 GR,8.9,tottus
1,PAN MOLDE INTEGRAL TOTTUS 600 GR,8.9,tottus
2,PAN MOLDE BLANCO ARTESANO BIMBO 530 GR,12.0,tottus
3,PAN MOLDE MARMOL SIN CORTE DON MAMINO 560 GR,12.2,tottus
4,PAN MOLDE BLANCO SIN CORTEZA DON MAMINO 560 GR,13.5,tottus
...,...,...,...
111,Pan de Molde Doble Fibra Bimbo Bolsa 420 g,11.2,metro
112,Pan de Molde de Mix de Cereales Mestemacher Bo...,14.9,metro
113,Pan de Molde con Semillas de Girasol Mestemach...,14.9,metro
114,Pan de Molde Integral con Frutas y Cereales Bi...,10.9,metro


In [191]:
new_df=df_total #copiamos nuestra tabla total en otra tabla
for i in filtros: #Lazo o bucle que arroja las filas de la tala total que contienen las palabras de la lista "filtros"
    a=i+"|"+i+"s"
    new_df=new_df[new_df["Nombre_producto"].str.contains(a,case=False , na=False,regex=True)]
new_df

,Nombre_producto,Precios_en_soles,Supermercado
2,PAN MOLDE BLANCO ARTESANO BIMBO 530 GR,12.0,tottus
12,PAN BIMBO MOLDE REBANADO FUENTE DE CALCIO 480 GR,10.0,tottus
15,PAN BLANCO BIMBO 750 GR,13.8,tottus
16,PAN DOBLE FIBRA BIMBO 420 GR,10.2,tottus
17,PAN INTEGRAL SIN CORTEZA BIMBO 650 GR,15.0,tottus
18,PAN BIMBO BLANCO CON SEMILLAS 600 GR,14.0,tottus
19,PAN INTEGRAL MULTICEREAL BIMBO 600 GR,14.0,tottus
22,PAN INTEGRAL EXTRA GRANDE BIMBO 750 GR,14.0,tottus
23,PAN MULTICEREAL VITAL BIMBO 360 GR,10.0,tottus
24,PAN ARTESANO INTEGRAL BIMBO 530 GR,12.0,tottus


In [195]:
len(new_df)

40

In [196]:
productos_1=new_df["Nombre_producto"] # Lista que contiene solo los nombres de los productos

In [197]:
list(productos_1)[0]

'PAN MOLDE BLANCO ARTESANO BIMBO 530 GR'

In [198]:
import difflib #libreria que halla la similiridad entre dos cadenas de caracteres 

In [199]:
porcentajes=[] #lista vacia que almacenara las similaridades en porcentaje
for i in productos_1:
  porcentaje=difflib.SequenceMatcher(isjunk=None , a=list(productos_1)[0], b=i ) #tomamos como producto de referencia al que se encuentra en la primera fila y hallamos su similaridad con los otros nombres de la tabla
  porcentaje=porcentaje.ratio()*100 #arroja el valor de la similitud en porcentaje  
  porcentaje=round(porcentaje,2) #redondeamos el porcentaje a 2 decimales. 
  porcentajes.append(porcentaje) #agregamos los valores de la similitud en la lista vacia





In [200]:
porcentajes

[100.0,
 55.81,
 72.13,
 63.64,
 66.67,
 56.76,
 50.67,
 50.0,
 58.33,
 69.44,
 69.44,
 60.32,
 45.45,
 64.79,
 63.77,
 32.18,
 31.58,
 27.59,
 36.14,
 32.18,
 29.89,
 27.59,
 32.18,
 27.59,
 29.27,
 26.47,
 20.0,
 28.57,
 32.0,
 25.58,
 25.58,
 25.58,
 25.58,
 23.76,
 27.27,
 27.91,
 30.23,
 25.74,
 27.5,
 18.18]

In [201]:
new_df["porcentajes"]=porcentajes #añadimos la lista "porcentaje" a nuestra tabla total

In [202]:
new_df

,Nombre_producto,Precios_en_soles,Supermercado,porcentajes
2,PAN MOLDE BLANCO ARTESANO BIMBO 530 GR,12.0,tottus,100.00
12,PAN BIMBO MOLDE REBANADO FUENTE DE CALCIO 480 GR,10.0,tottus,55.81
15,PAN BLANCO BIMBO 750 GR,13.8,tottus,72.13
16,PAN DOBLE FIBRA BIMBO 420 GR,10.2,tottus,63.64
17,PAN INTEGRAL SIN CORTEZA BIMBO 650 GR,15.0,tottus,66.67
18,PAN BIMBO BLANCO CON SEMILLAS 600 GR,14.0,tottus,56.76
19,PAN INTEGRAL MULTICEREAL BIMBO 600 GR,14.0,tottus,50.67
22,PAN INTEGRAL EXTRA GRANDE BIMBO 750 GR,14.0,tottus,50.00
23,PAN MULTICEREAL VITAL BIMBO 360 GR,10.0,tottus,58.33
24,PAN ARTESANO INTEGRAL BIMBO 530 GR,12.0,tottus,69.44


In [203]:
precio_A_compararar=list(new_df["Precios_en_soles"])[0] #tomamos como precio de referencia del producto a la primera fila de la tabla new_df

In [204]:
precio_A_compararar

12.0

In [205]:
super_mercados

['metro', 'plazavea', 'tottus']

In [206]:
for i in super_mercados: 
  df=new_df[(new_df["Precios_en_soles"]>=precio_A_compararar-3) &  (new_df["Precios_en_soles"]<=precio_A_compararar+3)  & #indicamos que nos arrojen los productos cuyo precio oscila entre 16.5 +- 2 soles
       (new_df["Supermercado"]==i)]
  max=df["porcentajes"].max() #sacamos el valor de similitud maximo de la tabla df
  if i=="metro": # En caso exista varias instancias que cumplen con la condicion del precio y sean del supermercado metro hacer lo siguiente:
    producto_metro=df[df["porcentajes"]==max] #extrae de todas las instancias, la que posee el maximo valor de similitud con el nombre del producto solicitado y la guarda en un nuevo dataframe
  if i=="plazavea":
    producto_plazavea=df[df["porcentajes"]==max]
  if i=="tottus":
    producto_tottus=new_df[new_df["porcentajes"]==max]





In [207]:
producto_metro

,Nombre_producto,Precios_en_soles,Supermercado,porcentajes
90,Pan de Molde Blanco Bimbo Bolsa 750 g,14.8,metro,32.0


In [208]:
producto_plazavea

,Nombre_producto,Precios_en_soles,Supermercado,porcentajes
49,Pan de Molde Blanco BIMBO Artesano Bolsa 530g,12.4,plazavea,36.14


In [209]:
producto_tottus

,Nombre_producto,Precios_en_soles,Supermercado,porcentajes
2,PAN MOLDE BLANCO ARTESANO BIMBO 530 GR,12.0,tottus,100.0


In [210]:
informacion=pd.concat([producto_metro,producto_plazavea,producto_tottus]) #Juntamos estos 3 dataframes en uno solo

In [211]:
lista_producto=informacion.sort_values(by="Precios_en_soles") #lo ordenamos en funcion del precio (de menos a más)

In [212]:
lista_producto # Dataframe final que se enviara por correo

,Nombre_producto,Precios_en_soles,Supermercado,porcentajes
2,PAN MOLDE BLANCO ARTESANO BIMBO 530 GR,12.0,tottus,100.00
49,Pan de Molde Blanco BIMBO Artesano Bolsa 530g,12.4,plazavea,36.14
90,Pan de Molde Blanco Bimbo Bolsa 750 g,14.8,metro,32.00


In [213]:
len(lista_producto)

3

In [214]:
frase="con un precio de"
mercado_1=""
mercado_2=""
mercado_3=""
precio_1=""
precio_2=""
precio_3=""
c_1=""
c_2=""
c_3=""
for i in range(len(lista_producto)):
  if i==0:
    mercado_1=list(lista_producto["Supermercado"])[i]
    precio_1=list(lista_producto["Precios_en_soles"])[i]
    c_1=frase
  if i==1:
    mercado_2=list(lista_producto["Supermercado"])[i]
    precio_2=list(lista_producto["Precios_en_soles"])[i]
    c_2=frase
  if i==2:
    mercado_3=list(lista_producto["Supermercado"])[i]
    precio_3=list(lista_producto["Precios_en_soles"])[i]
    c_3=frase




In [133]:
mercado_1

'plazavea'

In [134]:
mercado_2

'tottus'

In [135]:
mercado_3

'metro'

In [136]:
precio_1

4.49

In [137]:
precio_2

4.6

In [138]:
precio_3

4.89

In [215]:
import csv 
import pandas as pd
L=pd.DataFrame(lista_producto) # nuevo dataframe que guardara todo lo solicitado  
L.to_csv("Productos.csv",encoding='UTF-8') #Guardamos el dataframe en un archivo csv de nombre "Productos"
print(L) 

                                  Nombre_producto  Precios_en_soles  \
2          PAN MOLDE BLANCO ARTESANO BIMBO 530 GR              12.0   
49  Pan de Molde Blanco BIMBO Artesano Bolsa 530g              12.4   
90          Pan de Molde Blanco Bimbo Bolsa 750 g              14.8   

   Supermercado  porcentajes  
2        tottus       100.00  
49     plazavea        36.14  
90        metro        32.00  


In [216]:
import smtplib, ssl # Nos permite realizar el envío/  # Brinda seguridad

from email import encoders
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# PEDIR DATOS PARA INICIAR SESIÓN
username = 'stereomendez@gmail.com'
password = 'sxkrbzykyzhahiyx'

asunto = 'INFORMACIÓN DE PRODUCTOS'

# CREAR MENSAJE
mensaje = MIMEMultipart("alternative")  # con esta variable que creamos ("a") se puede definir cosas como el emisor y el receptor
mensaje["Subject"] = asunto                     # El Subject viene hacer asunto            
mensaje["From"] = username                     # El from viene hacer el emisor
mensaje["To"] = destinatario                    # El To viene hacer el receptor
 
html = f"""
<html>
<boby>
      Hola {destinatario}<br>
      el precio para el {producto}  en orden descendente para los supermecados es :<br>
      {mercado_1} {c_1} {precio_1},<br>
      {mercado_2} {c_2} {precio_2},<br>
      {mercado_3} ´{c_3} {precio_3},<br>
</boby>
</html>
"""
# EL CONTENIDO DEL MENSAJE COMO HMTL
parte_html = MIMEText(html, "html")

# AGREGAR ESE CONTENIDO AL MENSAJE
mensaje.attach(parte_html)

archivo = "Productos.csv"

with open(archivo, "rb") as adjunto: # GOZU
    contenido_adjunto = MIMEBase("application", "octet-stream")
    contenido_adjunto.set_payload(adjunto.read())

encoders.encode_base64(contenido_adjunto)

contenido_adjunto.add_header(
    "Content-Disposition",
    f"attachment; filename= {archivo}",
)

mensaje.attach(contenido_adjunto)
text = mensaje.as_string()

context = ssl.create_default_context()
# Envíar correo, tenemos que definir el servidor,puerto y luego utilizar el contexto
with smtplib.SMTP_SSL("smtp.gmail.com", 465, context = context) as server:
    server.login(username, password)                              #  Estamos logeando al correo
    server.sendmail(username, destinatario, text)          # Enviamos el correo / el as_stringl lo que hace es trasformar este objeto('a') a una cadena de texto